In [ ]:
!pip install torch torchvision

In [ ]:
import warnings
warnings.simplefilter("ignore")
import os
import torch
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from PIL import Image, ImageDraw

In [ ]:
# Load pre-trained Faster R-CNN model
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Define a function for image annotation
def annotate_image(image_path, output_path='annotated_images', confidence_threshold=0.5):
    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # Process each image in the directory
    for filename in os.listdir(image_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            # Load the image
            img_path = os.path.join(image_path, filename)
            img = Image.open(img_path).convert("RGB")

            # Preprocess the image
            transform = T.Compose([T.ToTensor()])
            input_img = transform(img).unsqueeze(0)

            # Perform inference
            with torch.no_grad():
                prediction = model(input_img)

            # Draw bounding boxes on the image
            draw = ImageDraw.Draw(img)
            boxes = prediction[0]['boxes'].cpu().numpy()
            scores = prediction[0]['scores'].cpu().numpy()

            # Create a list to store coordinates
            coordinates = []

            for box, score in zip(boxes, scores):
                if score > confidence_threshold:
                    draw.rectangle(box, outline="red", width=3)
                    coordinates.append(box.tolist())

            # Save annotated image
            output_img_path = os.path.join(output_path, f'annotated_{filename}')
            img.save(output_img_path)

            # Save coordinates to a text file
            output_txt_path = os.path.join(output_path, f'coordinates_{filename}.txt')
            with open(output_txt_path, 'w') as f:
                for coord in coordinates:
                    f.write(','.join(map(str, coord)) + '\n')

# Test the annotation function on a directory of images
annotate_image('C:/Users/shubham.kamble/Pictures/Auto_Annoatatepoc/image_dataset', output_path='C:/Users/shubham.kamble/Pictures/Auto_Annoatatepoc/output_images', confidence_threshold=0.5)
